# 00 - Preassembling files for anndata object


In [ ]:
import pandas as pd
from scipy.sparse import coo_matrix
from scipy.io import mmwrite

# --- 1. ATAC-Seq-Daten laden ---
atac_data = pd.read_csv("/mnt/workspace_stud/napkon_data/wp1_atac/combined_atac_napkon_5_upeak_1perc_all.matrix.raw.anno", sep="\t")  # Peak-Matrix
patient_metadata = pd.read_csv("/mnt/workspace_stud/napkon_data/wp1_atac/out.txt", sep="\t")  # Patienten-Metadaten

# --- 2. Dateien vorbereiten ---
# 2.1. Features-Datei (Peaks + Metadaten)
features = atac_data.iloc[:, 0:24] #Auswahl der ersten 24 Spalten -> die Peak ids und Spalten mit Metadaten 
features.to_csv("/mnt/workspace_stud/stud4/stud4/atac_analysis/adatas/anndata_0/regions.tsv", sep="\t", index=False, header=False)

# 2.2. Barcodes-Datei (Patienten + Metadaten)
# Patienten-IDs aus ATAC-Daten extrahieren
patient_ids = atac_data.columns[24:]  # Patienten-Spalten starten ab Spalte 25 -> ab index 24
barcodes = pd.DataFrame({
    "atac id": patient_ids #Benennung der Spalte in barcodes.tsv mit "atac id"
}).merge(patient_metadata, left_on="atac id", right_on="atac id", how="left")  # Metadaten anfügen
barcodes.to_csv("/mnt/workspace_stud/stud4/stud4/atac_analysis/adatas/anndata_0/barcodes.tsv", sep="\t", index=False, header=False)

# --- 3. Count-Matrix erstellen ---
# Zählwerte extrahieren und NaN durch 0 ersetzen
counts = atac_data.iloc[:, 24:].fillna(0)

# Sparse-Matrix erstellen
sparse_matrix = coo_matrix(counts.values)

# MTX-Datei speichern
mmwrite("/mnt/workspace_stud/stud4/stud4/atac_analysis/adatas/anndata_0/matrix.mtx", sparse_matrix)